# Projeto de Previsão de Demanda e Análise de sazionalidade - Marketplace

## Objetivo do Projeto
Identificar padrões sazonais e de performance nos produtos mais rentáveis, utilizando análise preditiva para orientar decisões estratégicas que visem um aumento de 15% na receita.

### Integração dos Relatórios

#### Relatório de Evolução do Negócio (3 abas)
Foco em comportamento dos compradores, sazonalidade e conversão

Métricas úteis:

- Visitas, compradores únicos, taxa de recompra

- Vendas por faixa de preço

- Vendas por tipo de comprador

#### 2. Relatório de Desempenho do Produto
Foco em performance mês a mês

Métricas úteis:

- Intenção de compra vs. conversão real

- Vendas brutas por mês

- Produtos com maior engajamento

#### 3. Relatório de desempenho das publicações
Foco na performance dos anúncios

Métricas úteis:

- Visitas únicas

- Quantidade de vendas

- Conversão de visitas em vendas

- Conversão de visitas em compradores

### Análise Preditiva e Estratégia de Receita

#### Etapas:
- Unificar os dados dos três relatórios com Pandas

- Criar métricas derivadas:

- Receita por produto/mês

- Conversão por faixa de preço

- Receita por tipo de comprador

- Identificar os produtos mais rentáveis

- Foco nos que têm alta margem e boa conversão

- Modelar a previsão de receita

- Usar regressão ou séries temporais (ARIMA, Prophet)

- Simular estratégias para +15% de receita

- Aumentar estoque nos meses de pico

- Investir em anúncios para faixas de preço mais lucrativas

- Focar em novos compradores com alta conversão

### 1. Importar os requisitos

In [ ]:
pip install --upgrade openpyxl


In [10]:
from openpyxl import load_workbook
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os



### 2. Limpar e transformar os dados
- Converter datas

- Preencher nulos

- Criar colunas derivadas (mês, semana, etc.)

#### ETL - Relatório de Evolução do negócio

#### Aba 1 – Evolução do Negócio

In [3]:
evolucao = pd.read_csv("Relatorio_evolucao_negocio_2025_01_01-2025_10_01.csv", encoding='latin1', sep=';', skiprows=5)
evolucao = evolucao.loc[:, ~evolucao.columns.str.contains('^Unnamed')]
evolucao.columns = evolucao.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)
evolucao['ms'] = pd.to_datetime(evolucao['ms'], format='%d/%m/%y', errors='coerce')


#### Aba 2 – Vendas por Faixa de Preço

In [5]:
faixa_preco = pd.read_csv("vendas_por_faixa_preco_2025.csv", encoding='latin1', sep=';', skiprows=4)
faixa_preco = faixa_preco.loc[:, ~faixa_preco.columns.str.contains('^Unnamed')]
faixa_preco.columns = faixa_preco.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)
# Não existe coluna 'ms' neste arquivo, então não é necessário converter datas aqui

#### Aba 3 – Vendas por Tipo de Comprador

In [6]:
tipo_comprador = pd.read_csv("vendas_por_tipo_comprador_2025.csv", encoding='latin1', sep=';', skiprows=4)
tipo_comprador = tipo_comprador.loc[:, ~tipo_comprador.columns.str.contains('^Unnamed')]
tipo_comprador.columns = tipo_comprador.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)


#### ETL – Relatório de Desempenho das Publicações

In [ ]:


csv_path = "Relatorio_desempenho_publicacoes_2025_01_01-2025_10_01.csv"
if os.path.exists(csv_path):
    publicacoes = pd.read_csv(csv_path, encoding='latin1', sep=';', skiprows=5)

    # Remover colunas Unnamed
    publicacoes = publicacoes.loc[:, ~publicacoes.columns.str.contains('^Unnamed')]

    # Padronizar nomes
    publicacoes.columns = publicacoes.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)

    # Converter colunas numéricas
    numericas = ['visitas_nicas', 'quantidade_de_vendas', 'compradores_nicos', 'unidades_vendidas', 'vendas_brutas_brl']
    for col in numericas:
        publicacoes[col] = pd.to_numeric(publicacoes[col], errors='coerce')
else:
    print(f"Arquivo não encontrado: {csv_path}. Verifique o caminho e o nome do arquivo.")


#### ETL – Relatório de Desempenho do Produto

In [16]:
produto = pd.read_csv("Relatorio_desempenho_do_produt_2025_01_01-2025_10_01.csv", encoding='latin1', sep=';', skiprows=5)

# Remover colunas Unnamed
produto = produto.loc[:, ~produto.columns.str.contains('^Unnamed')]

# Padronizar nomes
produto.columns = produto.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('[^a-z0-9_]', '', regex=True)

# Converter coluna de mês para datetime (usando 'ms')
produto['ms'] = pd.to_datetime(produto['ms'], format='%B', errors='coerce')

# Converter colunas numéricas
numericas = ['visitas_nicas', 'total_de_visitas', 'curtir', 'visitas_nicas_com_inteno_de_compra',
             'unidades_com_inteno_de_compra', 'quantidade_de_intenes_de_compra', 'valor_bruto_de_inteno_de_compra_brl',
             'quantidade_de_vendas', 'compradores_nicos', 'unidades_vendidas', 'vendas_brutas_brl']
for col in numericas:
    produto[col] = pd.to_numeric(produto[col], errors='coerce')


### 1. Evolução Mensal do Negócio

In [35]:
# Receita bruta por mês
receita_mensal = evolucao.groupby(evolucao['ms'].dt.month)['vendas_brutas'].sum()

# Taxa de recompra média por mês
recompra_mensal = evolucao.groupby(evolucao['ms'].dt.month)['taxa_de_recompra'].mean()

# Conversão de visitas em vendas por mês
conversao_mensal = evolucao.groupby(evolucao['ms'].dt.month)['converso'].mean()

# Cancelamentos e devoluções
cancelamentos = evolucao.groupby(evolucao['ms'].dt.month)['valor_de_vendas_canceladas'].sum()
devolucoes = evolucao.groupby(evolucao['ms'].dt.month)['valor_de_vendas_devolvidas'].sum()



### 2. Vendas por Faixa de Preço

In [ ]:

# Faixas com maior receita
faixa_receita = faixa_preco.sort_values(by='vendas_brutas', ascending=False)

# Faixas com melhor conversão
faixa_conversao = faixa_preco.sort_values(by='converso', ascending=False)

# Perfil de compradores por faixa
faixa_preco['_de_compradores'] = (faixa_preco['_de_compradores'].astype(str).str.replace('%','').str.replace(',','.').astype(float))



### 3. Vendas por Tipo de Comprador

In [57]:
# Novos vs. recorrentes
tipo_comprador['vendas_brutas'] = pd.to_numeric(tipo_comprador['vendas_brutas'], errors='coerce')
tipo_comprador['converso'] = (
    tipo_comprador['converso']
    .astype(str)
    .str.replace('%','')
    .str.replace(',','.')
    .astype(float)
)


# Participação nas vendas
tipo_comprador['_de_participao_nas_vendas'] = (
    tipo_comprador['_de_participao_nas_vendas']
    .astype(str)
    .str.replace('%','')
    .str.replace(',','.')
    .astype(float)
)

tipo_comprador.sort_values(by='_de_participao_nas_vendas', ascending=False, inplace=True)


### 4. Desempenho por Produto


In [60]:
# Renomear coluna corretamente
produto.rename(columns={'quantidade_de_vendas': 'vendas_brutasbrl'}, inplace=True)


In [67]:
# Intenção de compra
produto['valor_bruto_de_inteno_de_compra_brl'] = pd.to_numeric(produto['valor_bruto_de_inteno_de_compra_brl'], errors='coerce')
produto['vendas_brutasbrl'] = pd.to_numeric(produto['vendas_brutasbrl'], errors='coerce')

# Conversão
produto['converso_de_visitas_em_inteno_de_compra'] = (
    produto['converso_de_visitas_em_inteno_de_compra']
    .astype(str)
    .str.replace('%','')
    .str.replace(',','.')
    .astype(float)
)

produto['converso_de_visitas_em_vendas'] = (
    produto['converso_de_visitas_em_vendas']
    .astype(str)
    .str.replace('%','')
    .str.replace(',','.')
    .astype(float)
)

# Gap entre intenção e venda
produto['gap_intencao_venda'] = produto['valor_bruto_de_inteno_de_compra_brl'] - produto['vendas_brutasbrl']


### 5. Desempenho por Publicação

In [71]:
# Conversão por anúncio
publicacoes['converso_de_visitas_em_vendas'] = publicacoes['converso_de_visitas_em_vendas'].str.replace('%','').str.replace(',','.').astype(float)
publicacoes['vendas_brutas_brl'] = pd.to_numeric(publicacoes['vendas_brutas_brl'], errors='coerce')

# Participação na receita total
publicacoes['participacao'] = publicacoes['vendas_brutas_brl'] / publicacoes['vendas_brutas_brl'].sum()

# Anúncios com maior impacto
publicacoes.sort_values(by='vendas_brutas_brl', ascending=False, inplace=True)


### 6. Simulação de Meta (+15%)

In [72]:
# Meta de crescimento
meta = publicacoes['vendas_brutas_brl'].sum() * 0.15
print(f"Meta de aumento: R${meta:,.2f}")

# Produtos com maior potencial de crescimento
publicacoes['potencial'] = publicacoes['converso_de_visitas_em_vendas'] * publicacoes['visitas_nicas']
publicacoes.sort_values(by='potencial', ascending=False, inplace=True)

# Estratégia por faixa de preço
faixa_preco['potencial_receita'] = faixa_preco['vendas_brutas'] * 1.15


Meta de aumento: R$0.00


### 7. Script para exportar os dados tratados

In [73]:


# Criar pasta de saída se não existir
os.makedirs("dados_tratados", exist_ok=True)

# Exportar cada DataFrame
evolucao.to_csv("dados_tratados/evolucao_negocio.csv", index=False)
faixa_preco.to_csv("dados_tratados/faixa_preco.csv", index=False)
tipo_comprador.to_csv("dados_tratados/tipo_comprador.csv", index=False)
produto.to_csv("dados_tratados/desempenho_produto.csv", index=False)
publicacoes.to_csv("dados_tratados/desempenho_publicacoes.csv", index=False)

print("✅ Exportação concluída! Arquivos salvos na pasta 'dados_tratados'.")


✅ Exportação concluída! Arquivos salvos na pasta 'dados_tratados'.


### 8. Simulação de Meta (+15%) — Exportação

In [75]:




# Ordenar por potencial de crescimento
publicacoes_meta = publicacoes.copy()
publicacoes_meta.sort_values(by='potencial', ascending=False, inplace=True)

# Exportar simulação
publicacoes_meta.to_csv("dados_tratados/simulacao_meta_15.csv", index=False)
